# Test notebook for Sx operations.

In [1]:
import servicex
from servicex import ServiceXDataset
from servicex.minio_adaptor import MinioAdaptor
from servicex.servicex_adaptor import ServiceXAdaptor
from func_adl_servicex import ServiceXDatasetSource
import uproot_methods
from numpy import genfromtxt
import qastle
import csv
import time
import math
import asyncio
import numpy as np

### Datasets:

localds://mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00

user.emmat:user.emmat.mc16_13TeV.311311.MadGraphPythia8EvtGen_A14NNPDF31LO_HSS_LLP_mH125_mS15.mc16d.200131.forsX_trees.root

data15_13TeV:data15_13TeV.AllYear.physics_Main.PhysCont.DAOD_TOPQ1.grp15_v01_p4173

In [14]:
    dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00")
    query = ServiceXDatasetSource(dataset) \
        .SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
        .Select('lambda j: (j.getAttributeFloat("LArQuality"))') \
        .AsPandasDF("JetMoment") \
        .value()

In [15]:
print(query)

        JetMoment
entry            
0        0.391937
1        0.000000
2        0.044899
3        0.019566
4        0.033224
...           ...
857133   0.000000
857134   0.000000
857135   0.000000
857136   0.000000
857137   0.000000

[11355980 rows x 1 columns]


In [2]:
# good dataset = ServiceXDataset('mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00')
# bad dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10201_r10210_p4180_tid21860364_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10724_r10726_p4180_tid21859882_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10724_r10726_p4180_tid21859885_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r9364_r9315_p4180_tid21859934_00')    
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_s3126_r10201_r10210_p4180_tid21860366_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r10724_r10726_p3978_tid19368338_00')
# bad dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r10724_r10726_p3978_tid19368354_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r9364_r9315_p3978_tid19370588_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT19.e3649_e5984_s3126_r10724_r10726_p3978_tid19511379_00')
# bad dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10201_r10210_p4180_tid21860364_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_SUSY18.e3649_e5984_s3126_r10201_r10210_p3840_tid18281770_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301001.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_180M250.deriv.DAOD_EXOT19.e3649_e5984_s3126_r10724_r10726_p3978_tid19509568_00')
dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.simul.HITS.e3649_e5984_s3126_tid12539189_00')

async def retrieve_jet_data(dataset):
    jets = ServiceXDatasetSource(dataset) \
        .Select('lambda e: e.Jets("AntiKt4EMTopoJets").Where(lambda jet: jet.pt()/1000>60)') \
        .Select('lambda e: (e.Select(lambda jet: jet.pt()), \
                            e.Select(lambda jet: jet.eta()), \
                            e.Select(lambda jet: jet.phi()))') \
        .AsAwkwardArray(("JetPt", "JetEta", "JetPhi")) \
        .value_async()

    return await jets

async def retrieve_ele_data(dataset):
    electrons = ServiceXDatasetSource(dataset) \
        .Select('lambda e: e.Electrons("Electrons")') \
        .Select('lambda e: (e.Select(lambda ele: ele.pt()), \
                            e.Select(lambda ele: ele.eta()), \
                            e.Select(lambda ele: ele.phi()))') \
        .AsAwkwardArray(("ElePt", "EleEta", "ElePhi")) \
        .value_async()

    return await electrons

data_jet, data_ele = await asyncio.gather(retrieve_jet_data(dataset), retrieve_ele_data(dataset))

print(data_jet[b'JetEta'])

CancelledError: 

In [17]:
print(data_jet[b'JetEta'])

[[] [-1.5031309127807617] [2.001312255859375 1.7870326042175293] ... [] [] [-1.2351728677749634 -1.7818306684494019]]


In [18]:
    data_jet, data_ele = await asyncio.gather(retrieve_jet_data(dataset), retrieve_ele_data(dataset))

    jetr = np.sqrt(data_jet[b'JetEta']**2 + data_jet[b'JetPhi']**2)
    eler = np.sqrt(data_ele[b'EleEta']**2 + data_ele[b'EleEta']**2)

In [13]:
print(eler)

[[] [0.9269971736989183 2.33096045392677] [2.3865369742471754 2.9326191918869413] ... [1.5386993370718418 1.5379411995608279 2.756536233716794 ... 0.5575009368662772 1.5379411995608279 0.5575009368662772] [1.6810007532235376 2.2618271469017444 0.27739989374603125] [2.299167063046274 2.205763577599947 0.17922829318081857 ... 2.1953725249806775 0.03880249344737656 0.13167630100922068]]


In [11]:
    jetr = np.sqrt(np.power(jetEta, 2) + np.power(jetPhi, 2))
    eler = np.sqrt(np.power(eleEta, 2) + np.power(elePhi, 2))

TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable sqrt method

In [19]:
np.sqrt(jetEta)

TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable sqrt method

In [26]:
test=np.array([1.0, 0.0, 1.5], dtype='float64')

In [27]:
np.sqrt(test)

array([1.        , 0.        , 1.22474487])

In [28]:
print(jetEta)

[array([], dtype=float64) array([-3.05267406]) array([], dtype=float64)
 ... array([], dtype=float64) array([-0.08107731])
 array([], dtype=float64)]


In [32]:
np.sqrt(np.ndarray.flatten(jetEta, order='C'))

TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable sqrt method

In [38]:
test=np.ndarray([2, 1])

In [39]:
print(test)

[[1.        ]
 [1.22474487]]


In [40]:
np.sqrt(test)

array([[1.        ],
       [1.10668192]])

In [41]:
np.type(jetEta)

AttributeError: module 'numpy' has no attribute 'type'

In [43]:
type(jetEta)

numpy.ndarray

In [2]:
# good dataset = ServiceXDataset('mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00')
# bad dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10201_r10210_p4180_tid21860364_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10724_r10726_p4180_tid21859882_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10724_r10726_p4180_tid21859885_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r9364_r9315_p4180_tid21859934_00')    
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_s3126_r10201_r10210_p4180_tid21860366_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r10724_r10726_p3978_tid19368338_00')
# bad dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r10724_r10726_p3978_tid19368354_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r9364_r9315_p3978_tid19370588_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT19.e3649_e5984_s3126_r10724_r10726_p3978_tid19511379_00')
# bad dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10201_r10210_p4180_tid21860364_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_SUSY18.e3649_e5984_s3126_r10201_r10210_p3840_tid18281770_00')
# good dataset = ServiceXDataset('mc16_13TeV:mc16_13TeV.301001.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_180M250.deriv.DAOD_EXOT19.e3649_e5984_s3126_r10724_r10726_p3978_tid19509568_00')

dataset_list = [ServiceXDataset('mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_SUSY18.e3649_e5984_s3126_r10201_r10210_p3840_tid18281770_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10724_r10726_p4180_tid21859882_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r10724_r10726_p4180_tid21859885_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_e5984_s3126_r9364_r9315_p4180_tid21859934_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT0.e3649_s3126_r10201_r10210_p4180_tid21860366_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r10724_r10726_p3978_tid19368338_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301001.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_180M250.deriv.DAOD_EXOT19.e3649_e5984_s3126_r10724_r10726_p3978_tid19509568_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT12.e3649_e5984_s3126_r9364_r9315_p3978_tid19370588_00'), \
               ServiceXDataset('mc16_13TeV:mc16_13TeV.301000.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYee_120M180.deriv.DAOD_EXOT19.e3649_e5984_s3126_r10724_r10726_p3978_tid19511379_00')]

In [3]:
    async def fetch_data(dataset):
        query = ServiceXDatasetSource(dataset) \
            .Select('lambda e: (e.Jets("AntiKt4EMTopoJets"), e.Electrons("Electrons"))') \
            .Select('lambda ls: (ls[0].Select(lambda jet: jet.e()), \
                                 ls[0].Select(lambda jet: jet.eta()), \
                                 ls[0].Select(lambda jet: jet.index()), \
                                 ls[0].Select(lambda jet: jet.m()), \
                                 ls[0].Select(lambda jet: jet.phi()), \
                                 ls[0].Select(lambda jet: jet.pt()), \
                                 ls[0].Select(lambda jet: jet.px()), \
                                 ls[0].Select(lambda jet: jet.py()), \
                                 ls[0].Select(lambda jet: jet.pz()), \
                                 ls[0].Select(lambda jet: jet.rapidity()), \
                                 ls[1].Select(lambda ele: ele.charge()), \
                                 ls[1].Select(lambda ele: ele.e()), \
                                 ls[1].Select(lambda ele: ele.eta()), \
                                 ls[1].Select(lambda ele: ele.index()), \
                                 ls[1].Select(lambda ele: ele.m()), \
                                 ls[1].Select(lambda ele: ele.nCaloClusters()), \
                                 ls[1].Select(lambda ele: ele.nTrackParticles()), \
                                 ls[1].Select(lambda ele: ele.phi()), \
                                 ls[1].Select(lambda ele: ele.pt()), \
                                 ls[1].Select(lambda ele: ele.rapidity()))') \
            .AsAwkwardArray(("JetE", \
                             "JetEta", \
                             "JetIndex", \
                             "JetM", \
                             "JetPhi", \
                             "JetPt", \
                             "JetPx", \
                             "JetPy", \
                             "JetPz", \
                             "JetRapidity", \
                             "EleCharge", \
                             "EleE", \
                             "EleEta", \
                             "EleIndex", \
                             "EleM", \
                             "EleNClusters", \
                             "EleNTrack", \
                             "ElePhi", \
                             "ElePt", \
                             "EleRapidity")) \
            .value_async()

        return await query


In [5]:
    data_list = []
    for i in range(10):
        data_slot = fetch_data(dataset_list[i])
        data_list.append(data_slot)

    ds0, ds1, ds2, ds3, ds4, ds5, ds6, ds7, ds8, ds9 = await asyncio.gather(data_list[0], \
                                                            data_list[1], \
                                                            data_list[2], \
                                                            data_list[3], \
                                                            data_list[4], \
                                                            data_list[5], \
                                                            data_list[6], \
                                                            data_list[7], \
                                                            data_list[8], \
                                                            data_list[9])

ServiceXException: (ServiceXException(...), 'ServiceX rejected the transformation request: (504)<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n<hr><center>nginx/1.15.9</center>\r\n</body>\r\n</html>\r\n')

In [18]:
    def convert_dataset(retrieved_data):
        retrieved_data = retrieved_data[b'JetPt']
        return len(retrieved_data)

In [25]:
    total_length = len(ds0[b'JetPt']) + len(ds1[b'JetPt']) + \
                   len(ds2[b'JetPt']) + len(ds3[b'JetPt']) + \
                   len(ds4[b'JetPt']) + len(ds5[b'JetPt']) + \
                   len(ds6[b'JetPt']) + len(ds7[b'JetPt']) + \
                   len(ds8[b'JetPt']) + len(ds9[b'JetPt'])

In [27]:
print(total_length)

10264173


In [13]:
async def test_lambda_capture():
    dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00")

    async def retrieve_jet_data(dataset):
        jets = ServiceXDatasetSource(dataset) \
            .Select('lambda e: e.Jets("AntiKt4EMTopoJets").Where(lambda jet: jet.pt()/1000>60)') \
            .Select('lambda j: (j.Select(lambda jet: jet.pt()), \
                                j.Select(lambda jet: jet.eta()), \
                                j.Select(lambda jet: jet.phi()))') \
            .AsAwkwardArray(("JetPt", "JetEta", "JetPhi")) \
            .value_async()

        return await jets

    async def retrieve_ele_data(dataset):
        electrons = ServiceXDatasetSource(dataset) \
            .Select('lambda e: e.Electrons("Electrons")') \
            .Select('lambda e: (e.Select(lambda ele: ele.pt()), \
                                e.Select(lambda ele: ele.eta()), \
                                e.Select(lambda ele: ele.phi()))') \
            .AsAwkwardArray(("ElePt", "EleEta", "ElePhi")) \
            .value_async()

        return await electrons

    data_jet, data_ele = await asyncio.gather(retrieve_jet_data(dataset), retrieve_ele_data(dataset))

    jetr = np.sqrt(data_jet[b'JetEta']**2 + data_jet[b'JetPhi']**2)
    eler = np.sqrt(data_ele[b'EleEta']**2 + data_ele[b'EleEta']**2)

    electrons_within_tolerance = []
    for electron in eler:
        event_counter = 0
        for jet in jetr:
            if abs(jet - electron) <= 1.0:
                electrons_within_tolerance.append(event_counter)
        event_counter += 1
    
    final_list = []

    for i in range(len(electrons_within_tolerance)):
        final_list.append(data_ele[b'ElePt'][electrons_within_tolerance[i]])
        
    return final_list

In [14]:
lambda_capture_list = test_lambda_capture()

f:\program files (x86)\python 3.7\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: coroutine 'test_lambda_capture' was never awaited
  """Entry point for launching an IPython kernel.


In [4]:
print(lambda_capture_list)

<coroutine object test_lambda_capture at 0x000002D3BB9D7AC8>


In [6]:
dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00")

def test_lambda_capture(dataset):
    
    def retrieve_jet_lambda(dataset):
        jets = ServiceXDatasetSource(dataset) \
            .Select('lambda e: (e.Jets("AntiKt4EMTopoJets").Where(lambda jet: jet.pt()/1000 > 60.0), e.Electrons("Electrons"))') \
            .Select('lambda ls: ls[0].Select(lambda jet: jet.pt())') \
            .AsAwkwardArray(('JetPt')) \
            .value()
        
        return jets
    
    lambda_test = retrieve_jet_lambda(dataset)
    print(len(lambda_test[b'JetPt']))
    

#                     .Where(lambda r: abs(np.sqrt(ls[1].Select(lambda ele: ele.eta()).Select(lambda eleta: eleta * eleta) \
#                                                             + ls[1].Select(lambda ele: ele.phi()).Select(lambda elphi: elphi * elphi)) \
#                                                         - np.sqrt(ls[0].Select(lambda jet: jet.eta()).Select(lambda jetet: jetet * jetet)  \
#                                                             + ls[0].Select(lambda jet: jet.phi()).Select(lambda jephi: jephi * jephi))) < 1.0 ) \
#                                      .Select(lambda ls: ls[0].Select(lambda jet: jet.pt()))') \

In [11]:
test_lambda_capture(dataset)

ServiceXException: (ServiceXException(...), 'ServiceX login request rejected: 404')

In [11]:
from servicex import ServiceXDataset
from func_adl_servicex import ServiceXDatasetSource


#dataset_uproot = "user.kchoi:user.kchoi.ttHML_80fb_ttbar"
dataset_uproot = "data15_13TeV:data15_13TeV.00282784.physics_Main.deriv.DAOD_PHYSLITE.r9264_p3083_p4165_tid21568807_00"
uproot_transformer_image = "sslhep/servicex_func_adl_uproot_transformer:issue6"

sx_dataset = ServiceXDataset(dataset_uproot, image=uproot_transformer_image)
ds = ServiceXDatasetSource(sx_dataset, "CollectionTree")
data = ds.Select("lambda e: {'AntiKt4EmTopoJetsAux': e.pt}") \
    .AsParquetFiles('junk.parquet') \
    .value()

print(data)

ServiceXException: (ServiceXException(...), 'Failed to transform all files in 1fc02739-ebc4-4948-873f-019a38bb16e5')

In [7]:
print(data)

[WindowsPath('C:/Users/dliu1/AppData/Local/Temp/servicex/data/9071d0a2-f877-449c-8ce9-eefc0fa8aeab/root___fax.mwt2.org_1094__pnfs_uchicago.edu_atlaslocalgroupdisk_rucio_user_kchoi_2c_31_ttHML_80fb_410472_mc16d.root.parquet'), WindowsPath('C:/Users/dliu1/AppData/Local/Temp/servicex/data/9071d0a2-f877-449c-8ce9-eefc0fa8aeab/root___fax.mwt2.org_1094__pnfs_uchicago.edu_atlaslocalgroupdisk_rucio_user_kchoi_50_af_ttHML_80fb_410472_mc16a.root.parquet'), WindowsPath('C:/Users/dliu1/AppData/Local/Temp/servicex/data/9071d0a2-f877-449c-8ce9-eefc0fa8aeab/root___fax.mwt2.org_1094__pnfs_uchicago.edu_atlaslocalgroupdisk_rucio_user_kchoi_6c_d7_ttHML_80fb_410470_mc16a.root.parquet'), WindowsPath('C:/Users/dliu1/AppData/Local/Temp/servicex/data/9071d0a2-f877-449c-8ce9-eefc0fa8aeab/root___fax.mwt2.org_1094__pnfs_uchicago.edu_atlaslocalgroupdisk_rucio_user_kchoi_75_6f_ttHML_80fb_410470_mc16d.root.parquet')]


In [3]:
import pandas as pd

In [8]:
import pandas as pd
pd.read_parquet(data)

,lep_pt_1,lep_pt_2
0,29697.210938,13235.856445
1,59942.164062,148132.250000
2,18633.767578,0.000000
3,10757.475586,115532.664062
4,78170.820312,0.000000
...,...,...
550278,25388.658203,0.000000
550279,18321.812500,0.000000
550280,22894.816406,0.000000
550281,28860.708984,0.000000


In [1]:
import uproot

file = uproot.open("root://tech-gftp.hep.technion.ac.il:1094//storage/atlas/atlasdatadisk/rucio/data15_13TeV/3f/96/DAOD_PHYSLITE.21568807._000107.pool.root.1")

ImportError: Install pyxrootd package with:
    conda install -c conda-forge xrootd
(or download from http://xrootd.org/dload.html and manually compile with cmake; setting PYTHONPATH and LD_LIBRARY_PATH appropriately).